# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- camelCase to snake_case if required
- gen_prime() is not used

### Import and Configuration

In [873]:
import sympy
import random
import coloredlogs, logging

In [874]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [875]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10

msg = 12345678900987654321

In [876]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))



# Voraussetzungen

In [877]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

attacker = {}

created = False
while not created:
    try:
        attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

INFO Attacker Key successfully generated!


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [878]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
user = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, attacker["E"] ,attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    user["P"] = P
    user["Q"] = Q

attacker["vP"] = vP
attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [324882239607288299194949213657925249187325562423797835761569847534279587980953013777783960352315086513893155555011207841926195233556297148395289209983763420368670606688825853725629767594517299272954559150747772777513025361705671662921398805520123403178359358435525772037244600569499279320537396789998724304637839250, 324882239607288299194949213657925249187325562423797835761569847534279587980953013777783960352315086513893155555011207841926195233556297148395289209983763420368670606688825853725629767594517299272954559150747772777513025361705671662921398805520123403178359358435525772037244600569499279320537396789998724304662674935] 	 P: 7238347394853575949602935516577307775305468714935605453708624210506743041124224080871754162137062392631380733602218925107688610451678047760020053727395370011525029463908481696618453789715755384674201870996145581510459250489906325821845038097775670961078277553748874585733458348690730376651156073988289 	 deltaB: 24835685
INFO Amount of failed atte

## Schritt 4

In [879]:
user["N"] = calc_n(user["P"], user["Q"])

## Schritt 5

In [880]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        user.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [881]:
cipher = pow(msg, user["E"], user["N"])
signature = pow(msg, user["D"], user["N"])

# Angriff

In [882]:
class Calc_Secret_Result():
    was_successfull = True
    key_parameter = {}

In [883]:
def calc_secrets(vP):
    calc_secret_result = Calc_Secret_Result()

    P = pow(vP, attacker["D"], attacker["N"])
    # Potential to optimize when N/P not hole Number
    Q =  int(pub_user["N"]//P)
    PhiN =  int(calc_phi_n(P, Q))
    try:
        D = modular_multiplicative_inverse(pub_user["E"], PhiN)
        calc_secret_result.key_parameter = {
        "P" : P,
        "Q" : Q,
        "PhiN" : PhiN,
        "D" : D, 
    }
    except ValueError:
        calc_secret_result.was_successfull = False
        logger.info("Trying to generate Key failed. Trying again ...")
    
    return calc_secret_result

In [884]:
def validate_through_signature(key_parameter):
    key_parameter["Sig"] = pow(msg, key_parameter["D"], collection_user["N"])
    if (key_parameter["Sig"] == signature):
        collection_user.update(key_parameter)
        return True
    return False
    

## Schritt 1

In [885]:
pub_user = {
    "N" : user["N"],
    "E" : user["E"]
}

collection_user = {
    "N" : pub_user["N"],
    "E" : pub_user["E"]
}

## Schritt 2

In [886]:
collection_user["vP"] = split_in_binary(user["N"])[0]
collection_user["vP'"] = collection_user["vP"]+1

In [887]:
secrets_for_vP = calc_secrets(collection_user["vP"])
secrets_for_vP_plus_one = calc_secrets(collection_user["vP'"])

if(secrets_for_vP.was_successfull and secrets_for_vP_plus_one.was_successfull):
    logger.info("Both key parameters of D were successfully calculated.")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
    elif(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull and not secrets_for_vP_plus_one.was_successfull):
    logger.error("Both key parameters calculated with errors!")
elif(not secrets_for_vP_plus_one.was_successfull):
    logger.info("Second key parameters calculated with error! First key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull):
    logger.info("First key parameters calculated with error! Second key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
else:
    logger.error("Logical programming error!")


INFO Both key parameters of D were successfully calculated.
INFO D' is the right privat Key: 649666717515129751808219633698741805390410644322073365914294801162076437868708460553027644133700530834533891645543521964353317906765661952125812191785439567527900931383740215508779493260510677248354020510429388854564317485001701264706681687033512989541968858965223973890920459147820488369410770622199303609928071025627000397327620513579760025897021148069649411581045658733197164322957112314794953690184743472163615935396513488508122993106650430479694502175514985219850170111581228569147622291024840435526249070425264041101122193267304518579418505318542194654639078296967996500401592451966631206833832382668210514945


# Result

In [888]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [889]:
# logger.info(sort_dic(attacker))
# logger.info(sort_dic(user))
# logger.info(sort_dic(collection_user))

# Validation Tests

### Validation test for vP

In [890]:
testVar = "vP"
altTestVar = "vP'"

if(collection_user[testVar] == attacker[testVar] or collection_user[altTestVar] == attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(attacker[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 13081267692974300404568690396662691591533690788544911975002122779552442741332048459678180090984816523005338375978187861058067512795443238045182407449208128987127733060914521514543566236203784878270741518478249743348230638936484037340430246737058094280634835491057155448402634049126835317916067578596130921737
INFO Current vP: 13081267692974300404568690396662691591533690788544911975002122779552442741332048459678180090984816523005338375978187861058067512795443238045182407449208128987127733060914521514543566236203784878270741518478249743348230638936484037340430246737058094280634835491057155448402634049126835317916067578596130921736 	 Current vP': 1308126769297430040456869039666269159153369078854491197500212277955244274133204845967818009098481652300533837597818786105806751279544323804518240744920812898712773306091452151454356623620378487827074151847824974334823063893648403734043024673705809428063483549105715544840263404912

### Validation test for P and P'

In [891]:
testVar = "P"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables P are in the expected state:
INFO Current P: 7238347394853575949602935516577307775305468714935605453708624210506743041124224080871754162137062392631380733602218925107688610451678047760020053727395370011525029463908481696618453789715755384674201870996145581510459250489906325821845038097775670961078277553748874585733458348690730376651156073988289


### Validation test for Q and Q'

In [892]:
testVar = "Q"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state!".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], attacker[testVar]))

INFO Success!!!
INFO The variables Q are in the expected state!
INFO Current Q: 324882239607288299194949213657925249187325562423797835761569847534279587980953013777783960352315086513893155555011207841926195233556297148395289209983763420368670606688825853725629767594517299272954559150747772777513025361705671662921398805520123403178359358435525772037244600569499279320537396789998724304643536869


### Validation test for PhiN and PhiN'

In [893]:
testVar = "PhiN"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables PhiN are in the expected state:
INFO Current PhiN: 2351610512695610510066372476909530196976556839306659930140475547868361190595184145945466261160014597527819861308516692180828207070968663685863855398434619279643247033318608949095159574196496395110239201002540182191873656248527284859572073148839617441018112630338909313943190957760420640999134761266270718700725552867410409888918288337887300375430062465547885898258151187245516496211267293871863423920809564399240130921083154458402642383498720572299739254353624664809880193220794024539027308856244844956764028325342152937506878924925313539083247265730497803186510466511560212684552243100780622819102745385151128201984


### Validation test for D and D'

In [894]:
testVar = "D"
altTestVar = "D'"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables D are in the expected state:
INFO Current D: 649666717515129751808219633698741805390410644322073365914294801162076437868708460553027644133700530834533891645543521964353317906765661952125812191785439567527900931383740215508779493260510677248354020510429388854564317485001701264706681687033512989541968858965223973890920459147820488369410770622199303609928071025627000397327620513579760025897021148069649411581045658733197164322957112314794953690184743472163615935396513488508122993106650430479694502175514985219850170111581228569147622291024840435526249070425264041101122193267304518579418505318542194654639078296967996500401592451966631206833832382668210514945


### Validation test for signature and signature'

In [895]:
testVar = "Sig"

if(collection_user[testVar] == signature):
    collection_user["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_user[testVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 2012533026719670336471083813922342887078023252894984322573515535244308031081529658558050899420109969617021137400059246506772334250849961538273825585477142560493909858989898278201535300540412304918768137159128296270427951854790935696026419248408778200351393775366826393014262482515957659455203306630131548995709865493909738496546085118984423151624632415319587008675829528589333387543170332671661796017723525654132401831759997928736022966401082716967183752281864479145938443786559962580138915323412608839759522750373082985180375221533147786241057198356704611061980132296440035146874903995681251893043723788504514631801
